# Export VE Data as DataFrame for LME Cue Analysis

In [2]:
import csv
import pandas as pd
import numpy as np
import os
import json

In [2]:
# residuals, clutter, complete ground plane, viewing duration, stimulus 

In [3]:
# dpath = '/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/verbal_judgement_analysis/data/finalVEMatched/z_scored/'

dpath = '/Users/pmahableshwarkar/Documents/Depth_Project/verbal_judgement_analysis/data/finalVEMatched/z_scored/'

# stim 
with open(dpath + 'z_final_stim_125.npy' , 'rb') as f:
    stim_125 = np.load(f, allow_pickle=True)
    
with open(dpath + 'z_final_stim_250.npy' , 'rb') as f:
    stim_250 = np.load(f, allow_pickle=True)

with open(dpath + 'z_final_stim_1000.npy' , 'rb') as f:
    stim_1000 = np.load(f, allow_pickle=True)
    
# residuals
with open(dpath + 'residuals/'+ 'z_residuals_125.npy' , 'rb') as f:
    residuals_125 = np.load(f, allow_pickle=True)
with open(dpath + 'residuals/'+ 'z_residuals_250.npy' , 'rb') as f:
    residuals_250 = np.load(f, allow_pickle=True) 
with open(dpath + 'residuals/'+ 'z_residuals_1000.npy' , 'rb') as f:
    residuals_1000= np.load(f, allow_pickle=True)
    
# difference between z-scored estimate and z-scored actual depth
with open(dpath + '/diff_125.npy' , 'rb') as f:
    diff_125 = np.load(f, allow_pickle=True)
with open(dpath + '/diff_250.npy' , 'rb') as f:
    diff_250 = np.load(f, allow_pickle=True) 
with open(dpath + '/diff_1000.npy' , 'rb') as f:
    diff_1000= np.load(f, allow_pickle=True)
    
# difference between raw estimate and raw actual depth
# with open('/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/verbal_judgement_analysis/data/finalVEMatched/raw' + '/raw_diff_125.npy' , 'rb') as f:
#     raw_diff_125 = np.load(f, allow_pickle=True)
# with open('/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/verbal_judgement_analysis/data/finalVEMatched/raw' + '/raw_diff_250.npy' , 'rb') as f:
#     raw_diff_250 = np.load(f, allow_pickle=True) 
# with open('/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/verbal_judgement_analysis/data/finalVEMatched/raw' + '/raw_diff_1000.npy' , 'rb') as f:
#     raw_diff_1000= np.load(f, allow_pickle=True)

In [4]:
path = '/Users/pmahableshwarkar/Documents/Depth_Project/SUN_scene_cue_coding/GP_size_dict.json'
# path = '/Users/prachimahableshwarkar/Documents/GW/Depth_MTurk/SUN_scene_cue_coding/GP_size_dict.json'

gp_size_dict = json.load(open(path))

# Find GP or VGP Outliers 

gpsize_list = [[key, gp_size_dict[key]] for key in gp_size_dict]
gpsize_mean = np.mean(np.array([elem[1] for elem in gpsize_list]))
gpsize_std = np.std(np.array([elem[1] for elem in gpsize_list]))

gpsize_outlierRange = [gpsize_mean - (3*gpsize_std), gpsize_mean + (3*gpsize_std)]

cleaned_gpsize_dict = {}
for key in gp_size_dict:
    if gpsize_outlierRange[0] < gp_size_dict[key] < gpsize_outlierRange[1]:
        cleaned_gpsize_dict[key] = gp_size_dict[key]
    else:
        print(key, 'excluded')
        
ordered_gp_size = []
gp_none = 0


for folder in [elem.split('/')[1][:-2] for elem in stim_250]:
    try:
        ordered_gp_size.append(cleaned_gpsize_dict[folder])
    except:
        gp_none += 1
        ordered_gp_size.append(None)

002118_2014-06-25_20-32-08_260595134347_rgbf000078-resize excluded
002657_2014-06-13_15-29-54_094959634447_rgbf000150-resize excluded


In [5]:
# Opening JSON file
f = open('/Users/pmahableshwarkar/Documents/Depth_Project/verbal_judgement_analysis/cue_analyses/cleaned_globalClutter.json')
  
# returns JSON object as 
# a dictionary
cleaned_globalClutter = json.load(f)

In [26]:
# cleaned_globalClutter


In [6]:
globalClutter = []

completeGP = []

trunc_residuals_125 = []
trunc_residuals_250 = []
trunc_residuals_1000 = []

stimulus = []

for stim in cleaned_globalClutter:
    if stim in list(cleaned_gpsize_dict.keys()):
        index_125 = [elem.split('/')[1][:-2] for elem in stim_125].index(stim)
        index_250 = [elem.split('/')[1][:-2] for elem in stim_250].index(stim)
        index_1000 = [elem.split('/')[1][:-2] for elem in stim_1000].index(stim)
        
        trunc_residuals_125.append(residuals_125[index_125])
        trunc_residuals_250.append(residuals_250[index_250])
        trunc_residuals_1000.append(residuals_1000[index_1000])
        
        globalClutter.append(cleaned_globalClutter[stim])
        completeGP.append(cleaned_gpsize_dict[stim])
        
        stimulus.append(stim)
        
    
len(trunc_residuals_125), len(trunc_residuals_250), len(trunc_residuals_1000), len(globalClutter), len(completeGP), len(stimulus)

(113, 113, 113, 113, 113, 113)

In [7]:
len([125] * len(trunc_residuals_125))

113

In [8]:
len(np.concatenate((np.abs(np.array(trunc_residuals_125)), np.abs(np.array(trunc_residuals_250)), np.abs(np.array(trunc_residuals_1000))), axis=0),)

339

In [9]:
# initialize data of lists.
data = {'residuals': trunc_residuals_125+trunc_residuals_250+trunc_residuals_1000,
        'abs_residuals': np.concatenate((np.abs(np.array(trunc_residuals_125)), np.abs(np.array(trunc_residuals_250)), np.abs(np.array(trunc_residuals_1000))), axis=0),
        'duration': [125] * len(trunc_residuals_125) + [250] * len(trunc_residuals_250) + [1000] * len(trunc_residuals_1000),
        'groundPlane': completeGP * 3,
        'clutter': globalClutter * 3,
        'stimulus': stimulus * 3}
  
# Create DataFrame
df = pd.DataFrame(data)
df

,residuals,abs_residuals,duration,groundPlane,clutter,stimulus
0,-0.099975,0.099975,125,8.082259,3,000866_2014-06-09_20-45-42_260595134347_rgbf00...
1,0.080464,0.080464,125,10.664824,19,003070_2014-06-15_14-58-27_094959634447_rgbf00...
2,-0.301428,0.301428,125,9.164038,11,001092_2014-06-15_17-34-58_260595134347_rgbf00...
3,-0.075657,0.075657,125,10.409246,14,002272_2014-06-28_18-53-56_260595134347_rgbf00...
4,-0.164808,0.164808,125,31.362143,23,000642_2014-06-08_16-59-25_260595134347_rgbf00...
...,...,...,...,...,...,...
334,0.084265,0.084265,1000,17.196771,12,001094_2014-06-15_17-36-00_260595134347_rgbf00...
335,0.127303,0.127303,1000,20.030652,24,001808_2014-06-26_20-50-58_260595134347_rgbf00...
336,0.341774,0.341774,1000,11.913177,14,000758_2014-06-08_22-05-08_260595134347_rgbf00...
337,0.322175,0.322175,1000,5.576512,7,000878_2014-06-08_22-52-45_260595134347_rgbf00...


In [10]:
np.std(np.array(df['residuals'])), np.std(np.array(df['duration'])), np.std(np.array(df['clutter'])), np.std(np.array(df['groundPlane']))


(0.3230816540755546, 386.40077064565435, 7.3032069249418825, 8.380988898546363)

In [11]:
# df.to_csv('data_for_lme.csv', index=False)

# Add Participant Residuals

In [34]:
p = '/Users/pmahableshwarkar/Documents/Depth_Project/verbal_judgement_analysis/data/finalVEMatched/z_scored/residuals/participant_residuals.csv'

p_data = pd.read_csv(p) 
p_stimuli = list(p_data.stimulus.unique())


In [35]:
incl_stimuli = stimulus

# images that we do not have ground plane and/or clutter data for 
excluded_stimuli = [x for x in p_stimuli if x not in incl_stimuli]

len(excluded_stimuli), 156-113

(43, 43)

In [36]:
# remove all rows that have data for images that we do not have ground plane and/or clutter data for
p_data = p_data[p_data.stimulus.isin(excluded_stimuli) == False]

len(p_data.stimulus.unique())

113

https://towardsdatascience.com/create-new-column-based-on-other-columns-pandas-5586d87de73d



In [37]:
def assignGroundPlane(row):  
    stim = row['stimulus']
    groundPlane = cleaned_gpsize_dict[stim]
    
    return groundPlane

def assignClutter(row):  
    stim = row['stimulus']
    clutter = cleaned_globalClutter[stim]
    
    return clutter


In [38]:
# add groundPlane data to the df by referencing the GP value for each image
p_data['groundPlane'] = p_data.apply(lambda row: assignGroundPlane(row), axis=1)


In [39]:
# add clutter data to the df by referencing the GP value for each image
p_data['clutter'] = p_data.apply(lambda row: assignClutter(row), axis=1)


In [47]:
cleaned_globalClutter['001808_2014-06-26_20-50-58_260595134347_rgbf000029-resize'], cleaned_gpsize_dict['001808_2014-06-26_20-50-58_260595134347_rgbf000029-resize']

(24, 20.030651550568866)

In [46]:
p_data.to_csv('participantData_for_lme.csv', index=False)